In [1]:
import pandas as pd
import numpy as np
import json
import os
import glob
from datetime import datetime, timedelta

pd.set_option('display.float_format', '{:.6f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# ============================================================
# CONFIGURATION
# ============================================================

# Platform: "Fyers" or "Dhan" or other platforms
Platform = "Fyers"

# Resolution: "D" (Day), "5S" (5 seconds), "1T" (1 minute), etc.
Resolution = "5S"

# Validate against TradingView (only for D resolution)
validate_with_tradingview = True if Resolution == "D" else False

# ============================================================

# Construct paths
base_path = r'D:\Programming\Download_Backtest_Deploy_data\1__Download'

if Platform == "Fyers":
    platform_base_path = os.path.join(base_path, '1__Download_data_Fyers_via_API')
    storage_path = os.path.join(platform_base_path, f'storage_Fyers_{Resolution}')
    nifty50_file = os.path.join(platform_base_path, f'NIFTY50_Fyers_{Resolution}.csv')
    file_suffix = f'_Fyers_{Resolution}.csv'
    filter_date = "2025-01-01"
elif Platform == "Dhan":
    platform_base_path = os.path.join(base_path, '1__Download_data_Dhan_via_API')
    storage_path = os.path.join(platform_base_path, f'storage_Dhan_{Resolution}')
    nifty50_file = os.path.join(platform_base_path, f'NIFTY50_Dhan_{Resolution}.csv')
    file_suffix = f'_Dhan_{Resolution}.csv'
    filter_date = "2025-01-01"
else:
    raise ValueError(f"Platform '{Platform}' not configured")

# Load corporate actions and component changes
json_file_path = os.path.join(base_path, 'TradingView_data_download', 'nifty50_data.json')
with open(json_file_path, 'r') as f:
    loaded_data = json.load(f)
nifty_50_components_list = loaded_data['nifty50_components']

# For validation (only D resolution)
if validate_with_tradingview:
    tv_nifty50_file = os.path.join(base_path, 'TradingView_data_download', 'NIFTY_50_TV_D.csv')
    nifty_50_reference_df = pd.read_csv(tv_nifty50_file)

print(f"Platform: {Platform}")
print(f"Resolution: {Resolution}")
print(f"Storage path: {storage_path}")
print(f"NIFTY 50 file: {nifty50_file}")
print(f"Validation: {validate_with_tradingview}")

Platform: Fyers
Resolution: 5S
Storage path: D:\Programming\Download_Backtest_Deploy_data\1__Download\1__Download_data_Fyers_via_API\storage_Fyers_5S
NIFTY 50 file: D:\Programming\Download_Backtest_Deploy_data\1__Download\1__Download_data_Fyers_via_API\NIFTY50_Fyers_5S.csv
Validation: False


In [3]:
# ============================================================
# STEP 1: Load NIFTY 50 file and all constituent stocks
# ============================================================

# Load NIFTY 50 data
nifty50_df = pd.read_csv(nifty50_file)
nifty50_df['Datetime'] = pd.to_datetime(nifty50_df['Datetime'])
nifty50_df = nifty50_df[nifty50_df['Datetime'] >= filter_date].reset_index(drop=True)

# Create base dataframe for volume calculation
nifty_50_components_df = nifty50_df[['Datetime']].copy()

# Load all constituent stock files from storage folder
all_files = glob.glob(os.path.join(storage_path, f'*{file_suffix}'))
print(f"Found {len(all_files)} stock files in {storage_path}")

for file in all_files:
    symbol = os.path.basename(file).replace(file_suffix, '')
    symbol_df = pd.read_csv(file)
    symbol_df['Datetime'] = pd.to_datetime(symbol_df['Datetime'])
    symbol_df = symbol_df[symbol_df['Datetime'] >= filter_date].reset_index(drop=True)
    
    # Rename Volume column before merge to avoid suffix issues
    symbol_df = symbol_df.rename(columns={'Volume': f'{symbol}_v'})
    nifty_50_components_df = nifty_50_components_df.merge(symbol_df[['Datetime', f'{symbol}_v']], on='Datetime', how='left')

df = nifty_50_components_df.copy()

# Remove problematic dates (if resolution is D)
if Resolution == "D":
    dates_to_remove = pd.to_datetime(['2024-07-26', '2024-01-09', '2023-11-12']).date
    df['only_date'] = df['Datetime'].dt.date
    df = df[~df['only_date'].isin(dates_to_remove)]
    df.drop(columns=['only_date'], inplace=True)
    df.reset_index(drop=True, inplace=True)

print(f"\nData shape after loading: {df.shape}")
df.head(3)

Found 62 stock files in D:\Programming\Download_Backtest_Deploy_data\1__Download\1__Download_data_Fyers_via_API\storage_Fyers_5S

Data shape after loading: (841500, 63)


,Datetime,ADANIENT_v,ADANIPORTS_v,APOLLOHOSP_v,ASIANPAINT_v,AXISBANK_v,BAJAJFINSV_v,BAJAJ_AUTO_v,BAJFINANCE_v,BEL_v,BHARTIARTL_v,BPCL_v,BRITANNIA_v,CIPLA_v,COALINDIA_v,DIVISLAB_v,DRREDDY_v,EICHERMOT_v,ETERNAL_v,GAIL_v,GRASIM_v,HCLTECH_v,HDFCBANK_v,HDFCLIFE_v,HEROMOTOCO_v,HINDALCO_v,HINDUNILVR_v,ICICIBANK_v,INDIGO_v,INDUSINDBK_v,INFY_v,IOC_v,ITCHOTELS_v,ITC_v,JIOFIN_v,JSWSTEEL_v,KOTAKBANK_v,LTIM_v,LT_v,M&M_v,MARUTI_v,MAXHEALTH_v,NESTLEIND_v,NTPC_v,ONGC_v,POWERGRID_v,RELIANCE_v,SBILIFE_v,SBIN_v,SHREECEM_v,SHRIRAMFIN_v,SUNPHARMA_v,TATACONSUM_v,TATAMOTORS_v,TATASTEEL_v,TCS_v,TECHM_v,TITAN_v,TRENT_v,ULTRACEMCO_v,UPL_v,WIPRO_v,YESBANK_v
0,2025-01-01 09:15:00,4444,6330,0,0,0,1240,3833,0,0,5420,0,0,0,13987,705,0,160,0,13139,420,2285,49828,3661,763,6322,1761,22780,3162,6753,6753,37422,NaN,64973,50388,1796,4252,2269,1712,1270,552,0,1792,25888,21983,13922,0,1009,0,0,0,0,0,0,0,4222,2069,0,0,168,2176,0,226727
1,2025-01-01 09:15:05,5448,4916,1473,2237,7099,643,7947,10090,35030,3246,23967,324,2828,0,0,0,0,31071,0,0,0,0,1834,0,0,0,0,0,0,0,0,NaN,0,2730,0,2947,356,0,2362,0,2381,30,0,11023,13986,32916,649,20755,39,4210,5149,1926,35801,76726,0,0,1060,1847,0,0,46040,0
2,2025-01-01 09:15:10,575,0,76,361,3145,0,0,2660,12481,263,5287,52,1685,7157,97,4610,130,4903,11553,341,1955,33334,452,2068,9528,1061,16477,1638,2186,2422,11778,NaN,139781,19373,3181,0,461,496,0,570,0,588,29598,12197,1341,0,158,0,0,0,0,0,0,0,2173,2068,0,0,172,2919,0,47686


In [4]:
# ============================================================
# STEP 2: Calculate NIFTY 50 volume with adjustments
# ============================================================

print("Starting volume calculation...")

# nifty_50_components_list already has '_v' suffix, so just use it directly
nifty_50_components_list_v = ['Datetime'] + nifty_50_components_list
df_v = df[nifty_50_components_list_v].copy()
df_v['total_volume'] = np.nan

print(f"Created df_v with shape: {df_v.shape}")

# Pre-process component changes and corporate actions by date for fast lookup
print("Pre-processing component changes and corporate actions...")
component_changes_by_date = {}
for action in loaded_data['nifty50_components_changes']:
    component_changes_by_date[action['effective_date']] = action

corporate_actions_by_date = {}
for ca_action in loaded_data['corporate_actions']:
    date = ca_action['effective_date']
    if date not in corporate_actions_by_date:
        corporate_actions_by_date[date] = []
    corporate_actions_by_date[date].append(ca_action)

print(f"Found {len(component_changes_by_date)} component change dates")
print(f"Found {len(corporate_actions_by_date)} corporate action dates")

# Get unique dates in descending order
print("Getting unique dates...")
unique_dates = df_v['Datetime'].dt.date.unique()
unique_dates = sorted(unique_dates, reverse=True)
print(f"Total unique dates to process: {len(unique_dates)}")

volume_cols = [col for col in df_v.columns if col not in ['Datetime', 'total_volume']]
print(f"Starting with {len(volume_cols)} volume columns")

# Process component changes and corporate actions by date first
for date_idx, current_date in enumerate(unique_dates):
    current_date_str = current_date.strftime('%Y-%m-%d')
    
    if date_idx % 10 == 0:
        print(f"Processing date {date_idx+1}/{len(unique_dates)}: {current_date_str}")
    
    # Apply component changes
    if current_date_str in component_changes_by_date:
        action = component_changes_by_date[current_date_str]
        print(f"\nComponent change on {current_date_str}: {action}")
        
        for stock_remove in action['added']:
            stock_remove_v = stock_remove if stock_remove.endswith('_v') else stock_remove + '_v'
            if stock_remove_v in volume_cols:
                volume_cols.remove(stock_remove_v)
        
        for stock_add in action['removed']:
            stock_add_v = stock_add if stock_add.endswith('_v') else stock_add + '_v'
            if stock_add_v not in volume_cols:
                volume_cols.append(stock_add_v)
                volume_cols = sorted(set(volume_cols))
                if stock_add_v in df.columns:
                    df_v[stock_add_v] = df[stock_add_v]
    
    # Apply corporate actions
    if current_date_str in corporate_actions_by_date:
        print(f"Applying corporate actions for {current_date_str}...")
        for ca_action in corporate_actions_by_date[current_date_str]:
            symbol = ca_action['symbol'] + "_v" if not ca_action['symbol'].endswith('_v') else ca_action['symbol']
            factor = 1.0
            
            if ca_action['action_type'] == "bonus":
                a, b = ca_action['ratio'].split(":")
                a, b = float(a), float(b)
                factor = 1 / ((a + b) / b)
            
            if ca_action['action_type'] == "stock_split":
                x, y = ca_action['ratio'].split(":")
                x, y = float(x), float(y)
                factor = 1 / (x / y)
            
            if ca_action['action_type'] == "demerger":
                factor = float(ca_action['factor'])
                if Platform == "Fyers":
                    factor = 1.0
            
            if ca_action['action_type'] == "rights":
                factor = float(ca_action['factor'])
            
            if ca_action['action_type'] == "custom":
                factor = float(ca_action['factor'])
            
            # Determine adjustment range
            if ca_action['action_type'] != "demerger":
                i_ch = -2
            else:
                i_ch = -1
            
            if ca_action['action_type'] == "custom":
                i_ch = 0
            
            # Apply adjustment
            if symbol in df_v.columns:
                datetime_str = current_date_str + " 09:15:00"
                matching_idx = df_v[df_v['Datetime'] == datetime_str].index
                
                if len(matching_idx) > 0:
                    print(f"  Applying factor {factor} to {symbol}...")
                    mask_date = df_v['Datetime'].iloc[matching_idx[0] + i_ch]
                    first_date_of_data = str(df_v['Datetime'].loc[df_v[symbol].first_valid_index()])[:-9]
                    mask_start_date = '2021-01-01' if '2021-01-01' >= first_date_of_data else first_date_of_data
                    mask = (df_v['Datetime'] >= mask_start_date) & (df_v['Datetime'] <= mask_date)
                    df_v.loc[mask, symbol] = df_v.loc[mask, symbol] * factor
                    
                    print(f"  Corporate action applied: {ca_action['symbol']} - {ca_action['action_type']} (factor: {factor})")
    
    # Calculate total volume for all rows of this date using VECTORIZED operations
    if date_idx % 10 == 0:
        print(f"  Calculating volume for {current_date_str}...")
    
    date_mask = df_v['Datetime'].dt.date == current_date
    
    # Special volume adjustments
    special_volume = 0.0
    
    if current_date_str in ['2025-04-08', '2025-04-09', '2025-04-11']:
        if 'ETERNAL_v' in df_v.columns:
            special_volume = -df_v.loc[date_mask, 'ETERNAL_v']
    
    if current_date_str in ['2023-07-31', '2023-08-01']:
        if 'TATACONSUM_v' in df_v.columns:
            special_volume = -df_v.loc[date_mask, 'TATACONSUM_v']
    
    # Vectorized sum across volume columns
    df_v.loc[date_mask, 'total_volume'] = df_v.loc[date_mask, volume_cols].sum(axis=1) + (special_volume if isinstance(special_volume, (int, float)) else special_volume.values)
    
    if date_idx % 10 == 0:
        rows_count = date_mask.sum()
        print(f"  Completed {rows_count} rows for {current_date_str}")

print(f"\n✓ Volume calculation completed")
print(f"Final data shape: {df_v.shape}")

Starting volume calculation...
Created df_v with shape: (841500, 52)
Pre-processing component changes and corporate actions...
Found 13 component change dates
Found 20 corporate action dates
Getting unique dates...
Total unique dates to process: 187
Starting with 50 volume columns
Processing date 1/187: 2025-09-30

Component change on 2025-09-30: {'effective_date': '2025-09-30', 'added': ['MAXHEALTH', 'INDIGO'], 'removed': ['HEROMOTOCO', 'INDUSINDBK'], 'factor': 1.0}
  Calculating volume for 2025-09-30...
  Completed 4500 rows for 2025-09-30
Processing date 11/187: 2025-09-16
  Calculating volume for 2025-09-16...
  Completed 4500 rows for 2025-09-16
Processing date 21/187: 2025-09-02
  Calculating volume for 2025-09-02...
  Completed 4500 rows for 2025-09-02
Applying corporate actions for 2025-08-26...
  Applying factor 0.5 to HDFCBANK_v...
  Corporate action applied: HDFCBANK - bonus (factor: 0.5)
Processing date 31/187: 2025-08-18
  Calculating volume for 2025-08-18...
  Completed 4

In [5]:
# ============================================================
# STEP 3: Validation (only for D resolution with TradingView)
# ============================================================

if validate_with_tradingview:
    # Prepare TradingView reference data
    nifty_50_reference_df['Datetime'] = pd.to_datetime(nifty_50_reference_df['Datetime'])
    nifty_50_reference_df = nifty_50_reference_df[nifty_50_reference_df['Datetime'] >= filter_date].reset_index(drop=True)
    
    # Merge with calculated data
    df_v = df_v.merge(nifty_50_reference_df[['Datetime', 'Volume']], on='Datetime', how='left', suffixes=('', '_TV'))
    
    matched_count = 0
    total_count = 0
    
    print("\n" + "="*100)
    print(f"{'Index':<6} | {'Date':<12} | {'Result':<12} | {'TradingView Volume':<20} | {'Calculated Volume':<20} | {'Difference'}")
    print("="*100)
    
    for i in range(len(df_v)):
        if pd.notna(df_v['Volume'].iloc[i]):
            total_count += 1
            diff = abs(round(df_v['total_volume'].iloc[i], 0) - df_v['Volume'].iloc[i])
            
            if diff <= 5:
                result = 'Matched'
                matched_count += 1
            else:
                result = 'NOT Matched'
                print(f"{i:<6} | {df_v['Datetime'].iloc[i].date().strftime('%Y-%m-%d'):<12} | {result:<12} | {df_v['Volume'].iloc[i]:>20.0f} | {round(df_v['total_volume'].iloc[i], 0):>20.0f} | {diff:>10.0f}")
    
    print("="*100)
    match_percentage = (matched_count / total_count * 100) if total_count > 0 else 0
    print(f"\n✓ Validation Summary: {matched_count}/{total_count} matched ({match_percentage:.2f}%)")
    
    if match_percentage >= 95:
        print("✓ Validation PASSED: Data quality is good")
    else:
        print("✗ Validation FAILED: Data quality issues detected")
else:
    # For lower resolution than Day, compare daily aggregated volume with TradingView
    print("\n" + "="*100)
    print(f"Validation for {Resolution} resolution (comparing daily aggregated volume with TradingView)")
    print("="*100)
    
    # Load TradingView reference data
    tv_nifty50_file = os.path.join(base_path, 'TradingView_data_download', 'NIFTY_50_TV_D.csv')
    nifty_50_reference_df = pd.read_csv(tv_nifty50_file)
    nifty_50_reference_df['Datetime'] = pd.to_datetime(nifty_50_reference_df['Datetime'])
    nifty_50_reference_df = nifty_50_reference_df[nifty_50_reference_df['Datetime'] >= filter_date].reset_index(drop=True)
    
    # Aggregate calculated volume by date
    df_v['Date'] = df_v['Datetime'].dt.date
    daily_aggregated = df_v.groupby('Date')['total_volume'].sum().reset_index()
    daily_aggregated['Date'] = pd.to_datetime(daily_aggregated['Date'])
    
    # Merge with TradingView data
    nifty_50_reference_df['Date'] = nifty_50_reference_df['Datetime'].dt.date
    nifty_50_reference_df['Date'] = pd.to_datetime(nifty_50_reference_df['Date'])
    
    comparison_df = daily_aggregated.merge(nifty_50_reference_df[['Date', 'Volume']], on='Date', how='left', suffixes=('_Calc', '_TV'))
    
    matched_count = 0
    total_count = 0
    
    print(f"\n{'Date':<12} | {'Result':<12} | {'TradingView Volume':<20} | {'Calculated Volume':<20} | {'Difference':<15} | {'Diff %'}")
    print("-"*100)
    
    for i in range(len(comparison_df)):
        if pd.notna(comparison_df['Volume'].iloc[i]):
            total_count += 1
            tv_vol = comparison_df['Volume'].iloc[i]
            calc_vol = comparison_df['total_volume'].iloc[i]
            diff = abs(calc_vol - tv_vol)
            diff_pct = (diff / tv_vol * 100) if tv_vol > 0 else 0
            
            if diff <= 5:
                result = 'Matched'
                matched_count += 1
            else:
                result = 'NOT Matched'
            
            print(f"{comparison_df['Date'].iloc[i].strftime('%Y-%m-%d'):<12} | {result:<12} | {tv_vol:>20.0f} | {calc_vol:>20.0f} | {diff:>15.0f} | {diff_pct:>6.2f}%")
    
    print("="*100)
    match_percentage = (matched_count / total_count * 100) if total_count > 0 else 0
    print(f"\n✓ Validation Summary: {matched_count}/{total_count} matched ({match_percentage:.2f}%)")
    
    if match_percentage >= 95:
        print("✓ Validation PASSED: Data quality is good")
    else:
        print("✗ Validation FAILED: Data quality issues detected")


Validation for 5S resolution (comparing daily aggregated volume with TradingView)

Date         | Result       | TradingView Volume   | Calculated Volume    | Difference      | Diff %
----------------------------------------------------------------------------------------------------
2025-01-01   | NOT Matched  |            154921938 |            154876238 |           45700 |   0.03%
2025-01-02   | NOT Matched  |            283200811 |            283108299 |           92512 |   0.03%
2025-01-03   | NOT Matched  |            312279515 |            312186307 |           93208 |   0.03%
2025-01-06   | NOT Matched  |            278061806 |            277986547 |           75259 |   0.03%
2025-01-07   | NOT Matched  |            262337253 |            262290725 |           46528 |   0.02%
2025-01-08   | NOT Matched  |            266375381 |            266329246 |           46135 |   0.02%
2025-01-09   | NOT Matched  |            273566395 |            269194664 |         4371731 |   1.60%


In [6]:
# ============================================================
# STEP 4: Update NIFTY 50 file with calculated volume
# ============================================================

# Update volume column in original dataframe
nifty50_df = pd.read_csv(nifty50_file)
nifty50_df['Datetime'] = pd.to_datetime(nifty50_df['Datetime'])

# Merge calculated volume
df_v_update = df_v[['Datetime', 'total_volume']].copy()
df_v_update.rename(columns={'total_volume': 'Volume'}, inplace=True)

# Update only matching datetime rows
nifty50_df = nifty50_df.merge(df_v_update, on='Datetime', how='left', suffixes=('_old', ''))
nifty50_df['Volume'] = nifty50_df['Volume'].fillna(nifty50_df['Volume_old'])
nifty50_df.drop(columns=['Volume_old'], inplace=True)
nifty50_df = nifty50_df[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Date']]

# Save updated file
nifty50_df.to_csv(nifty50_file, index=False)

print(f"\n✓ NIFTY 50 file updated successfully: {nifty50_file}")
print(f"Total rows updated: {len(df_v_update)}")

# Display sample of updated data
print("\nSample of updated data:")
nifty50_df[nifty50_df['Datetime'] >= filter_date].head(5)


✓ NIFTY 50 file updated successfully: D:\Programming\Download_Backtest_Deploy_data\1__Download\1__Download_data_Fyers_via_API\NIFTY50_Fyers_5S.csv
Total rows updated: 841500

Sample of updated data:


,Datetime,Open,High,Low,Close,Volume,Date
0,2025-01-01 09:15:00,23637.650000,23679.900000,23633.350000,23678.200000,256538.000000,2025-01-01
1,2025-01-01 09:15:05,23678.450000,23681.700000,23664.850000,23670.500000,352045.000000,2025-01-01
2,2025-01-01 09:15:10,23668.900000,23670.000000,23659.600000,23663.250000,281274.000000,2025-01-01
3,2025-01-01 09:15:15,23662.800000,23662.800000,23653.600000,23655.100000,251249.000000,2025-01-01
4,2025-01-01 09:15:20,23656.100000,23657.850000,23647.500000,23653.400000,117163.000000,2025-01-01
